# Building centroid model

In [5]:
%config IPCompleter.use_jedi = False
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import numpy as np
from moffat import Moffat2D

## The model

![](../docs/architecture.png)

In [8]:
size = 15

n = Moffat2D(size)

model = models.Sequential([
    layers.Conv2D(64, (3, 3), activation='relu', input_shape=(size, size, 1), use_bias=True, padding="same"),
    layers.MaxPooling2D((2, 2), padding="same"),
    layers.Conv2D(128, (3, 3), activation='relu', use_bias=True, padding="same"),
    layers.MaxPooling2D((2, 2), padding="same"),
    layers.Conv2D(256, (3, 3), activation='relu', use_bias=True, padding="same"),
    layers.Flatten(),
    layers.Dense(2048, activation="sigmoid", use_bias=True),
    layers.Dense(512, activation="sigmoid", use_bias=True),
    layers.Dense(2),
])

### Training strategy

In [10]:
lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
  0.001,
  decay_steps=20,
  decay_rate=1,
  staircase=False)

model.compile(
    optimizer=tf.keras.optimizers.Adam(lr_schedule), 
    loss=tf.keras.losses.Huber(), 
    metrics=[tf.keras.metrics.RootMeanSquaredError()])

In [11]:
# Training

train = 2000
test = 10000
epochs = 80

train_dataset = n.random_model_label(train)
test_dataset = n.random_model_label(test)
train_dataset = tf.data.Dataset.from_tensor_slices(train_dataset)
test_dataset = tf.data.Dataset.from_tensor_slices(test_dataset)

BATCH_SIZE = 100
SHUFFLE_BUFFER_SIZE = 100

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

model.fit(train_dataset, epochs=epochs, validation_data=test_dataset, )

Epoch 1/80
20/20 [==============================] - 5s 189ms/step - loss: 3.0100 - root_mean_squared_error: 4.2488 - val_loss: 0.9404 - val_root_mean_squared_error: 1.6556
Epoch 2/80
20/20 [==============================] - 3s 171ms/step - loss: 0.5720 - root_mean_squared_error: 1.2166 - val_loss: 0.2140 - val_root_mean_squared_error: 0.6945
Epoch 3/80
20/20 [==============================] - 3s 168ms/step - loss: 0.1541 - root_mean_squared_error: 0.5824 - val_loss: 0.0905 - val_root_mean_squared_error: 0.4368
Epoch 4/80
20/20 [==============================] - 3s 167ms/step - loss: 0.0796 - root_mean_squared_error: 0.4078 - val_loss: 0.0602 - val_root_mean_squared_error: 0.3512
Epoch 5/80
20/20 [==============================] - 3s 165ms/step - loss: 0.0546 - root_mean_squared_error: 0.3338 - val_loss: 0.0447 - val_root_mean_squared_error: 0.3008
Epoch 6/80
20/20 [==============================] - 3s 166ms/step - loss: 0.0402 - root_mean_squared_error: 0.2849 - val_loss: 0.0343 - val_

KeyboardInterrupt: 

In [ ]:
# Saving
model.save()